<h2> <span style="color:teal"> Facebook posts connector</span> </h2>

Description:
- implement a fcb connector using Python 3 and the Meta Graph API
- our objective is to fetch posts from business pages i own
- the collected posts are filterd by a given topic(death of Jacques Chirac) and stored in MongoDB

* NB: due to fcb's privacy policy it is not possible to collect posts from all public pages so i will use a personal page


Project steps:
- necessary libraries importation
- load the following envirement variables in a .env file:
    * ACCESS_TOKEN=fcb_token(from the app on Graph API Explorer)
    * PAGE_ID=id(from the app on Graph API Explorer)
    * MONGODB_URI=mongodb://localhost:27017
- connect to MongoDB
- collect posts from private fcb page , we will need the following parameters:
    * page_id:
    * token
    * topic
    * limit number of posts to retrieve    
- save posts to MongoDB
- execute the connector

In [2]:
pip install requests pymongo python-dotenv

Note: you may need to restart the kernel to use updated packages.


<h4>1. Import required libraries</h4>

In [1]:
from dotenv import load_dotenv
from pymongo import MongoClient
import requests
import os

<h4>2.Load environment variables</h4>

In [2]:
load_dotenv()

ACCESS_TOKEN = os.getenv("ACCESS_TOKEN")
PAGE_ID = os.getenv("PAGE_ID")
MONGODB_URI = os.getenv("MONGODB_URI")

<h4>3.Connect to MongoDB</h4>

In [3]:
client = MongoClient(MONGODB_URI)
db = client["facebook_data"]
collection = db["posts"]

<h4>4.Collect posts from fcb page</h4>

In [4]:
import requests

def get_facebook_posts(page_id, token, topic=None, limit=10):
    url = f"https://graph.facebook.com/v18.0/{page_id}/posts"
    params = {
        "access_token": token,
        "fields": "message,full_picture,created_time,comments{message,from}",
        "limit": limit
    }

    response = requests.get(url, params=params)
    data = response.json()
    
    posts = data.get("data", [])
    filtered_posts = []

    for post in posts:
        message = post.get("message", "")
        if topic is None or topic.lower() in message.lower():
            filtered_posts.append({
                "message": message,
                "image": post.get("full_picture"),
                "created_time": post.get("created_time"),
                "comments": [
                    {"user": c["from"]["name"], "text": c["message"]} 
                    for c in post.get("comments", {}).get("data", [])
                ] if post.get("comments") else []
            })
    
    return filtered_posts


<h4>5. Save posts to MongoDB</h4>

In [5]:
def save_to_mongo(posts):
    if posts:
        collection.insert_many(posts)
        print(f"{len(posts)} posts saved to MongoDB.")
    else:
        print("No matching posts found.")


<h4>6. Execute the connector</h4>

In [6]:
topic = "Jacques Chirac"

posts = get_facebook_posts(PAGE_ID, ACCESS_TOKEN, topic=topic)
save_to_mongo(posts)


No matching posts found.
